# Generating names dataset

Here we will generate names dataset. Names dataset is supposed to be list of names.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import re
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [ ]:
file_lists=['/notebooks/nlp_deeplearning/charmodel/data/first_names.all.txt']

names_list = []
with open(file_lists[0],'r') as file:
    for name in file.read().splitlines()[1:]:
        filtered_name = re.sub(r'\W+', '', name)
        names_list.append(filtered_name.upper())

In [ ]:
names_list[:5]

['AISHA', 'AISHAH', 'AJAY', 'AAISHA', 'AAISHAH']

## Load data

In [ ]:
import sys
sys.path.insert(0,'/notebooks/Projects/Seq2Seq')

In [ ]:
from mllib.seq2seq.namegen import *
from dotmap import DotMap
from mllib.seq2seq.model import *
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers.neptune import NeptuneLogger
import pytorch_lightning as pl



In [ ]:
dsrc = get_dataset(names_list)


# Modelling

In [ ]:
hparams = DotMap({'vocab_size': len(dsrc.vocab), 
          'embedding_size': 30,
          'hidden_size': 300,
            'max_len': 15,
            'num_layers':2,
            'lr': 0.02})


# Training

In [ ]:

neptune_logger = NeptuneLogger(
    api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIwYWY0OTQ4MS03MGY4LTRhNjUtOTFlZC0zZjVjMjlmZGQxNjQifQ==",
    project_name="puneetgirdhar.in/charnn")

tensorboard_logger = TensorBoardLogger("tb_logs", name="my_model")

NeptuneLogger will work in online mode


In [ ]:
dls = dsrc.dataloaders(after_item=after_item, before_batch=pad_input_chunk_new, bs=32, n_inp=2)

# make sure that we use serializing option to instantiate the model

model = RNN(hparams, char2tensor = str(dict(dls.numericalize.o2i)), vocab=str(dls.numericalize.vocab))

checkpoint_callback = ModelCheckpoint(
    dirpath = './checkpoints',
    filename='{epoch}',
    save_top_k=3,
    monitor='val_loss',
    mode='min'
)


trainer = pl.Trainer(fast_dev_run=False, logger=neptune_logger, auto_lr_find='learning_rate',gpus=1,
                    callbacks=[EarlyStopping(monitor='val_loss',patience=5), checkpoint_callback],
                    )

GPU available: True, used: True
TPU available: None, using: 0 TPU cores


In [ ]:
trainer.fit(model, dls.train, dls.valid)

https://app.neptune.ai/puneetgirdhar.in/charnn/e/CHAR-33



  | Name      | Type             | Params
-----------------------------------------------
0 | dropout   | Dropout          | 0     
1 | embedding | Embedding        | 3.1 K 
2 | rnn       | LSTM             | 1.1 M 
3 | decoder   | Linear           | 31.3 K
4 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.621     Total estimated model params size (MB)


{'A': 'AÎहÜÍxxrepxxfakeĢĲहYPशFसÐ', 'B': 'BxxfakeĚŻĖVxxfldŐĂŅĞSČÔÃxxunk', 'R': 'RPŻĲWxxbosZŤÏÔRJYयCF', 'KAR': 'KARxxrepSŁĖZÁÖŽVĂÏxxupÜरŇ', 'TE': 'TEĲČÊŽĖŐÊVAĪÈÐश', 'CHRI': 'CHRIJŤŪOxxrepHŞŤAĂबÜxxunkÌÚ'}


{'A': 'AĚYAH', 'B': 'BBE', 'R': 'REMIE', 'KAR': 'KARLYN', 'TE': 'TEYON', 'CHRI': 'CHRILKEB'}


{'A': 'AरN', 'B': 'BEAGAN', 'R': 'RÁTZ', 'KAR': 'KARTO', 'TE': 'TEF', 'CHRI': 'CHRINER'}


...

{'A': 'AYSHU', 'B': 'BEK', 'R': 'RNTĞÐUR', 'KAR': 'KARLEYSA', 'TE': 'TEFACHO', 'CHRI': 'CHRISTIANA'}


{'A': 'AŞ', 'B': 'BYGALL', 'R': 'RŢĞRÍÐUR', 'KAR': 'KARPA', 'TE': 'TE', 'CHRI': 'CHRISSE'}


{'A': 'AमFRYD', 'B': 'BBÂKA', 'R': 'RÚDS', 'KAR': 'KARLO', 'TE': 'TESWARAN', 'CHRI': 'CHRISTIANJAMES'}


{'A': 'A', 'B': 'BJÖRN', 'R': 'RलD', 'KAR': 'KARSHAN', 'TE': 'TERIE', 'CHRI': 'CHRISTALIN'}


{'A': 'AHN', 'B': 'BJÖRT', 'R': 'RÓD', 'KAR': 'KARLISHA', 'TE': 'TERMAN', 'CHRI': 'CHRISTOPHER'}


# Evaluation

Now, we can generate some names randomly

In [ ]:
trainer.gen_name("A")

'AÓÁजÖÞXxxupxxfakexxwrepHĻÚĻÌÝ'

In [ ]:
train_from_scratch(names_list, hparams)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
